In [1]:
import pandas as pd
import numpy as np
import json
import requests
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup as bs
import base64
import datetime
import ticketpy

%matplotlib inline

# Ticketmaster Data

In [ ]:
api_keys = pd.read_csv('api_keys.csv')
# git ignore csv for pushing to repo
# .ipynb_checkpoints

In [ ]:
# tm_key = api_keys.loc[api_keys['endpoint'] == 'ticketmaster_endpoint', 'key'].values[0]

In [ ]:
# tm_endpoint = 'https://app.ticketmaster.com/discovery/v2/events.json?'

In [ ]:
# tm_params = {'ver':2.0,'feedtype':'json','api_key':tm_key}

In [ ]:
# test = requests.get(tm_endpoint,tm_params)
# test

In [2]:
events_request = requests.get('https://app.ticketmaster.com/discovery/v2/events.json?apikey=)
events_request

<Response [200]>

In [3]:
# assigning json file to variable
events = events_request.json()
events

{'_embedded': {'events': [{'name': 'Cleveland Browns vs. Pittsburgh Steelers',
    'type': 'event',
    'id': 'vv1AAZA06GkdNls4W',
    'test': False,
    'url': 'https://www.ticketmaster.com/cleveland-browns-vs-pittsburgh-steelers-cleveland-ohio-01-03-2021/event/05005907E3AF157D',
    'locale': 'en-us',
    'images': [{'ratio': '4_3',
      'url': 'https://s1.ticketm.net/dam/a/1c9/da3e7214-1e85-406b-af16-0222c4d421c9_1325111_CUSTOM.jpg',
      'width': 305,
      'height': 225,
      'fallback': False},
     {'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/1c9/da3e7214-1e85-406b-af16-0222c4d421c9_1325111_TABLET_LANDSCAPE_16_9.jpg',
      'width': 1024,
      'height': 576,
      'fallback': False},
     {'ratio': '3_2',
      'url': 'https://s1.ticketm.net/dam/a/1c9/da3e7214-1e85-406b-af16-0222c4d421c9_1325111_ARTIST_PAGE_3_2.jpg',
      'width': 305,
      'height': 203,
      'fallback': False},
     {'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/1c9/da3e721

In [4]:
events['page']

{'size': 20, 'totalElements': 77015, 'totalPages': 3851, 'number': 0}

In [5]:
# taking the events dictionary and pulling out desired items via for loop, assigning to list "event info",
# then turning that into a dataframe 

event_info = []
for event in events.get('_embedded',{}).get('events',{}):
     event_info.append({
        'TM_id': str(event.get('id',{})),
        'name' : str(event.get('name',{})),
        'artist': list(str(attraction.get('name')) for attraction in event['_embedded'].get('attractions',{})),
        'venue' : list(str(venue.get('name')) for venue in event['_embedded'].get('venues',{}))[0],
        'city' : list(str(venue.get('city',{}).get('name')) for venue in event['_embedded'].get('venues',{}))[0],
        'state' : list(str(venue.get('state',{}).get('stateCode')) for venue in event['_embedded'].get('venues',{}))[0],
        'date' : str(event.get('dates',{}).get('start',{}).get('dateTime',{})),
        'prices': event.get('priceRanges'),
        'genre' : event.get('classifications'),
     })
tmDF = pd.DataFrame(event_info)

In [6]:
tmDF

,TM_id,name,artist,venue,city,state,date,prices,genre
0,vv1AAZA06GkdNls4W,Cleveland Browns vs. Pittsburgh Steelers,"[Cleveland Browns, Pittsburgh Steelers]","FirstEnergy Stadium, Home of the Cleveland Browns",Cleveland,OH,2021-01-03T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
1,G5vVZpf7AtYVy,Magic Of Lights: Drive-Through Holiday Lights ...,[Magic Of Lights],Northwell Health at Jones Beach Theater,Wantagh,NY,{},"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
2,G5viZp2MCy3RR,Tennessee Titans vs. Detroit Lions,"[Tennessee Titans, Detroit Lions]",Nissan Stadium,Nashville,TN,2020-12-20T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
3,vv1AOv_8IGA5_vos,Magic Of Lights: Drive-Through Holiday Lights ...,[Magic Of Lights],PNC Bank Arts Center,Holmdel,NJ,{},"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
4,vvG1VZpb5yxMW6,Toronto Raptors vs. Miami Heat : Pre-Season Game,"[Toronto Raptors, Miami Heat]",Amalie Arena,Tampa,FL,2020-12-19T00:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
5,vvG1fZp2NIIo2P,Indianapolis Colts vs. Houston Texans,"[Indianapolis Colts, Houston Texans]",Lucas Oil Stadium,Indianapolis,IN,2020-12-20T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
6,vvG1bZp2BMoOze,Kansas City Chiefs vs. Atlanta Falcons,"[Kansas City Chiefs, Atlanta Falcons]",Arrowhead Stadium,Kansas City,MO,2020-12-27T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
7,vv1AAZAPJGkec3ZTV,Disney On Ice presents Dream Big,[Disney On Ice presents Dream Big],Bankers Life Fieldhouse,Indianapolis,IN,2021-01-15T00:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
8,vvG1bZp2BM4ZzC,Kansas City Chiefs vs. Los Angeles Chargers,"[Kansas City Chiefs, Los Angeles Chargers]",Arrowhead Stadium,Kansas City,MO,2021-01-03T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
9,G5viZp961eoMw,Disney On Ice presents Dream Big,[Disney On Ice presents Dream Big],Mississippi Coast Coliseum,Biloxi,MS,2021-01-09T20:30:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."


In [7]:
#removing brackets from given columns (no more list?)
tmDF['artist'] = tmDF['artist'].str[0]
tmDF['prices'] = tmDF['prices'].str[0]
tmDF['genre'] = tmDF['genre'].str[0]

In [8]:
tmDF

,TM_id,name,artist,venue,city,state,date,prices,genre
0,vv1AAZA06GkdNls4W,Cleveland Browns vs. Pittsburgh Steelers,Cleveland Browns,"FirstEnergy Stadium, Home of the Cleveland Browns",Cleveland,OH,2021-01-03T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
1,G5vVZpf7AtYVy,Magic Of Lights: Drive-Through Holiday Lights ...,Magic Of Lights,Northwell Health at Jones Beach Theater,Wantagh,NY,{},"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
2,G5viZp2MCy3RR,Tennessee Titans vs. Detroit Lions,Tennessee Titans,Nissan Stadium,Nashville,TN,2020-12-20T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
3,vv1AOv_8IGA5_vos,Magic Of Lights: Drive-Through Holiday Lights ...,Magic Of Lights,PNC Bank Arts Center,Holmdel,NJ,{},"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
4,vvG1VZpb5yxMW6,Toronto Raptors vs. Miami Heat : Pre-Season Game,Toronto Raptors,Amalie Arena,Tampa,FL,2020-12-19T00:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
5,vvG1fZp2NIIo2P,Indianapolis Colts vs. Houston Texans,Indianapolis Colts,Lucas Oil Stadium,Indianapolis,IN,2020-12-20T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
6,vvG1bZp2BMoOze,Kansas City Chiefs vs. Atlanta Falcons,Kansas City Chiefs,Arrowhead Stadium,Kansas City,MO,2020-12-27T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
7,vv1AAZAPJGkec3ZTV,Disney On Ice presents Dream Big,Disney On Ice presents Dream Big,Bankers Life Fieldhouse,Indianapolis,IN,2021-01-15T00:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
8,vvG1bZp2BM4ZzC,Kansas City Chiefs vs. Los Angeles Chargers,Kansas City Chiefs,Arrowhead Stadium,Kansas City,MO,2021-01-03T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."
9,G5viZp961eoMw,Disney On Ice presents Dream Big,Disney On Ice presents Dream Big,Mississippi Coast Coliseum,Biloxi,MS,2021-01-09T20:30:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS..."


In [9]:
# exploding out "TM_FV_prices" and "genre" to get just min/max price as separate columns, using "flatten" function 
# to draw out individual elements from nested list/dictionary within "TM_FV_prices" column

def flatten(x):
    d = {}
    if x is not None:
       # Each element of the dict
        for k,v in x.items():
          # Check value type
            if isinstance(v,list) and v[0] is not None:
             # If list: iter sub dict
                for k_s, v_s in v[0].items(): 
                    d["{}_{}".format(k, k_s)] = v_s
            else: d[k] = v
    return pd.Series(d)

In [10]:
print(tmDF['genre'].apply(flatten))

    primary                                            segment  \
0      True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
1      True  {'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...   
2      True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
3      True  {'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...   
4      True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
5      True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
6      True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
7      True  {'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...   
8      True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
9      True  {'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...   
10     True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
11     True     {'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}   
12     True  {'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...   
13     True  {'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...   
14     Tru

In [11]:
tmDF = tmDF.join(tmDF['genre'].apply(flatten),how='left', lsuffix='_left', rsuffix='_right')
tmDF

,TM_id,name,artist,venue,city,state,date,prices,genre_left,primary,segment,genre_right,subGenre,type,subType,family
0,vv1AAZA06GkdNls4W,Cleveland Browns vs. Pittsburgh Steelers,Cleveland Browns,"FirstEnergy Stadium, Home of the Cleveland Browns",Cleveland,OH,2021-01-03T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}","{'id': 'KnvZfZ7vAdE', 'name': 'Football'}","{'id': 'KZazBEonSMnZfZ7vFE1', 'name': 'NFL'}","{'id': 'KZAyXgnZfZ7v7l1', 'name': 'Group'}","{'id': 'KZFzBErXgnZfZ7vA7d', 'name': 'Team'}",False
1,G5vVZpf7AtYVy,Magic Of Lights: Drive-Through Holiday Lights ...,Magic Of Lights,Northwell Health at Jones Beach Theater,Wantagh,NY,{},"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...","{'id': 'KnvZfZ7vAeE', 'name': 'Fairs & Festiva...","{'id': 'KZazBEonSMnZfZ7vF1F', 'name': 'Undefin...","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
2,G5viZp2MCy3RR,Tennessee Titans vs. Detroit Lions,Tennessee Titans,Nissan Stadium,Nashville,TN,2020-12-20T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}","{'id': 'KnvZfZ7vAdE', 'name': 'Football'}","{'id': 'KZazBEonSMnZfZ7vFE1', 'name': 'NFL'}","{'id': 'KZAyXgnZfZ7v7l1', 'name': 'Group'}","{'id': 'KZFzBErXgnZfZ7vA7d', 'name': 'Team'}",False
3,vv1AOv_8IGA5_vos,Magic Of Lights: Drive-Through Holiday Lights ...,Magic Of Lights,PNC Bank Arts Center,Holmdel,NJ,{},"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...","{'id': 'KnvZfZ7vAeE', 'name': 'Fairs & Festiva...","{'id': 'KZazBEonSMnZfZ7vF1F', 'name': 'Undefin...","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
4,vvG1VZpb5yxMW6,Toronto Raptors vs. Miami Heat : Pre-Season Game,Toronto Raptors,Amalie Arena,Tampa,FL,2020-12-19T00:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}","{'id': 'KnvZfZ7vAde', 'name': 'Basketball'}","{'id': 'KZazBEonSMnZfZ7vFJA', 'name': 'NBA'}","{'id': 'KZAyXgnZfZ7v7l1', 'name': 'Group'}","{'id': 'KZFzBErXgnZfZ7vA7d', 'name': 'Team'}",False
5,vvG1fZp2NIIo2P,Indianapolis Colts vs. Houston Texans,Indianapolis Colts,Lucas Oil Stadium,Indianapolis,IN,2020-12-20T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}","{'id': 'KnvZfZ7vAdE', 'name': 'Football'}","{'id': 'KZazBEonSMnZfZ7vFE1', 'name': 'NFL'}","{'id': 'KZAyXgnZfZ7v7l1', 'name': 'Group'}","{'id': 'KZFzBErXgnZfZ7vA7d', 'name': 'Team'}",False
6,vvG1bZp2BMoOze,Kansas City Chiefs vs. Atlanta Falcons,Kansas City Chiefs,Arrowhead Stadium,Kansas City,MO,2020-12-27T18:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7nE', 'name': 'Sports'}","{'id': 'KnvZfZ7vAdE', 'name': 'Football'}","{'id': 'KZazBEonSMnZfZ7vFE1', 'name': 'NFL'}","{'id': 'KZAyXgnZfZ7v7l1', 'name': 'Group'}","{'id': 'KZFzBErXgnZfZ7vA7d', 'name': 'Team'}",False
7,vv1AAZAPJGkec3ZTV,Disney On Ice presents Dream Big,Disney On Ice presents Dream Big,Bankers Life Fieldhouse,Indianapolis,IN,2021-01-15T00:00:00Z,"{'type': 'standard', 'currency': 'USD', 'min':...","{'primary': True, 'segment': {'id': 'KZFzniwnS...",True,"{'id': 'KZFzniwnSyZfZ7v7n1', 'name': 'Miscella...","{'id': 'KnvZfZ7v7lI', 'name': 'Ice Shows'}","{'id': 'KZazBEonSMnZfZ7vAv6', 'name': 'Ice Sho...","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ',

In [ ]:
print(tmDF['prices'].apply(flatten))

# StubHub Data

In [ ]:
access_token = 
stubhub_endpoint = 'https://api.stubhub.com/sellers/search/bulk/events/v3'
headers = {'Authorization': 'Bearer ' + access_token, 'Accept': 'application/json', 'Accept-Encoding':'gzip'}

In [ ]:
stubhub_request = requests.get(stubhub_endpoint,headers = headers)
stubhub_request

In [ ]:
stubhub_request.json().keys()